In [36]:
import os, sys

sys.path.append('../') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [37]:
child_pid_missing = []
hiv_disclosures = [HIVDisclosureStatusA, HIVDisclosureStatusB, HIVDisclosureStatusC]
for disclosure_cls in hiv_disclosures:
    missing = disclosure_cls.objects.filter(
        Q(associated_child_identifier='') | Q(associated_child_identifier__isnull=True))
    child_pid_missing.extend(list(missing))

In [38]:
for missing in child_pid_missing:
    onschedule_model_cls = missing.maternal_visit.schedule.onschedule_model_cls
    try:
        onschedule_obj = onschedule_model_cls.objects.get(
            subject_identifier=missing.subject_identifier)
    except onschedule_model_cls.DoesNotExist:
        print('wheet!!')
    else:
        child_subject_identifier = onschedule_obj.child_subject_identifier
        missing.associated_child_identifier = child_subject_identifier
        missing.save_base(raw=True)